In [43]:
import sys

sys.path.append("../")

import sqlite3
import pandas as pd
from functions.env import DB_SCIENCE_PATH_NEW

data_path = "/Users/charlesdedampierre/Desktop/cultura_project/cultura.db"

conn = sqlite3.connect(data_path)
new_conn = sqlite3.connect(DB_SCIENCE_PATH_NEW)


In [48]:
df_ind = pd.read_sql("SELECT * FROM individual_occupations", conn)

df_ind_science = df_ind[df_ind["occupations_category"].str.contains("science")]
list_individuals = list(set(df_ind_science.individual_wikidata_id))

df_ind = df_ind[df_ind['individual_wikidata_id'].isin(list_individuals)]

print(len(set(df_ind.individual_wikidata_id)))

df_ind[df_ind['occupations_name']=='philosopher']


42668


,individual_wikidata_id,individual_name,occupations_wikidata_id,occupations_name,occupations_category
179,Q675,André-Marie Ampère,Q4964182,philosopher,writer
1674,Q201221,Alexander Ivanovich Herzen,Q4964182,philosopher,writer
1819,Q947485,"Victor de Riqueti, marquis de Mirabeau",Q4964182,philosopher,writer
2162,Q298860,Hippasus,Q4964182,philosopher,writer
2166,Q94840288,David Asher,Q4964182,philosopher,writer
...,...,...,...,...,...
318643,Q166876,Isidore of Seville,Q4964182,philosopher,writer
318691,Q1685927,Jean-Guillaume-César-Alexandre-Hippolyte Colins,Q4964182,philosopher,writer
318853,Q15970301,Julie Favre,Q4964182,philosopher,writer
319225,Q6934061,Mulla Mahmud Jaunpuri,Q4964182,philosopher,writer


In [45]:
df_ind_regions = pd.read_sql("SELECT * FROM individuals_regions", conn)
df_ind_regions = df_ind_regions.drop("individual_name", axis=1)
df_ind_regions = pd.merge(df_ind_regions, df_ind, on="individual_wikidata_id")
df_ind_regions = df_ind_regions[
    ["individual_wikidata_id", "individual_name", "region_code", "region_name"]
].drop_duplicates()

print(len(set(df_ind_regions.individual_wikidata_id)))

df_ind_regions.to_sql("individuals_regions", new_conn, if_exists="replace", index=False)

31089


109580

In [46]:
df_ind_birth = pd.read_sql("SELECT * FROM individuals_main_information", conn)
df_ind_birth = df_ind_birth[["individual_wikidata_id", "birthyear"]]
df_ind_birth["productive_year"] = df_ind_birth["birthyear"] + 35

df_ind_final = pd.merge(df_ind, df_ind_birth, on="individual_wikidata_id")

print(len(set(df_ind_final.individual_wikidata_id)))

df_ind_final.to_sql(
    "individuals_occupation_information", new_conn, if_exists="replace", index=False
)


42668


76865